In [1]:
### Run this notebook using cell -> run all
### Demo of one solution to 'blocking' until a widget value has changed

In [2]:
import time
import sys

class CaptureExecution:
    def __init__(self):
        self.captured_events = []
        self.shell = get_ipython()
        self.kernel = self.shell.kernel
        
    def step(self):
        self.kernel.do_one_iteration() # same thing an eventloop like %gui asyncio would do
        time.sleep(.01)
    
    def capture_event(self, stream, ident, parent):
        self.captured_events.append((stream, ident, parent))

    def start_capturing(self):
        "Overwrite the kernel shell handlers"
        self.kernel.shell_handlers['execute_request'] = self.capture_event

    def stop_capturing(self):
        "rever the kernel shell handler functions to their defaults"
        self.kernel.shell_handlers['execute_request'] = self.kernel.execute_request
    
    def replay_captured_events(self):
        sys.stdout.flush()
        for stream, ident, parent in self.captured_events:
            self.kernel.set_parent(ident, parent)
            self.kernel.execute_request(stream, ident, parent)
        self.captured_events = []
    
    def __enter__(self):
        self.start_capturing()
        self.shell.execution_count += 1
        
    def __exit__(self, *args):
        self.stop_capturing()
        self.replay_captured_events()

        
def capture_requests(widget=None, break_func=None):
    if widget is None and break_func is None:
        raise Exception("need to pass in a widget to watch to change or a break function to run")
    ctx = CaptureExecution()
    with ctx:
        while True:
            if widget is not None:
                if widget.value:
                    break
            else:
                if break_func():
                    break
            ctx.step()

In [10]:
import ipywidgets as widgets

w = widgets.Dropdown(options=['', 'foo', 'bar'])
    
display(w)

capture_requests(w)

Dropdown(options=('', 'foo', 'bar'), value='')

In [8]:
print(w.value)

foo


In [9]:
# this should get captured in a cell -> run all 
print("hello world")

hello world
